# Prescriptive Models Assignment 2

## Setup
First import the necessary python modules and libraries.

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

## 1. Hospital admission & quality of service
*Download health data.csv and load it into python. These are data from hospital admissions for coronary
artery bypass graft (CABG) in the UK. Among other things, you observe whether the patient died after the
surgery (coded up as patient died dummy), which hospital the patient visited (hospital id), and a series
of patient characteristics such as gender and age.*

In [8]:
df_1 = pd.read_csv('health_data.csv')
df_1.head()

,patient_id,hospital_id,admin_year,patient_died_dummy,startage,female_dummy
0,1,D,2003,0,81,0
1,2,H,2003,1,67,0
2,3,A,2003,0,54,0
3,4,E,2003,0,81,0
4,5,H,2003,0,69,0


### Question 1a.
*Based on the regression output, interpret the coefficients on the constant term and the dummy for
hospital D.*

In [11]:
model_hospital = smf.ols(formula = 'patient_died_dummy ~ hospital_id',data = df_1)
res_hospital = model_hospital.fit()
print(res_hospital.summary())

                            OLS Regression Results                            
Dep. Variable:     patient_died_dummy   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     119.3
Date:                Thu, 27 Jan 2022   Prob (F-statistic):          1.75e-220
Time:                        09:36:30   Log-Likelihood:                -7416.5
No. Observations:               24480   AIC:                         1.485e+04
Df Residuals:                   24470   BIC:                         1.493e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.0970      0.006  

The coefficient on the constant term is equal to 0.0970 which represents the patient mortality rate at hospital A following CABG surgery. In addition, the coefficient on hospital D is 0.1882 which indicates an increase in patient mortality rate when compared to hospital A, with a patient mortality rate of 0.0970 + 0.1882 = 0.2852. When compared to all other hospitals and their coefficient values, it seems that hospital D has the highest mortality rate following CABG surgery.

### Question 1b.
*What is the difference between the mortality rates at hospitals D and E (use the regression output to
derive this)?*

As we saw before, hospital D has a mortality rate of 0.2852. We can derive hospital E's mortality rate as 0.0970 - 0.0531 = 0.0439 which seems to be the lowest mortality rate among the hospitals. Therefore, the difference in mortality rates between hospitals D and E (the highest and lowest mortality rates respectively) is equal to 0.2852 - 0.0439 = 0.2413.

### Question 2a.
*Explain why the difference in mortality rate implied by this regression cannot be interpreted as the
causal effect of visiting a different hospital (i.e., the change in risk of dying when moving a patient
from hospital A to B cannot be inferred from this regression).*

Begin by creating a new dataframe based on only data for hospital A and B.

In [26]:
df_AB = df_1[df_1['hospital_id'].isin(['A','B'])]
df_AB.head()

,patient_id,hospital_id,admin_year,patient_died_dummy,startage,female_dummy
2,3,A,2003,0,54,0
6,7,A,2003,0,59,0
12,13,B,2003,0,65,0
15,16,A,2003,0,61,0
19,20,A,2003,0,71,0


In [27]:
model_AB = smf.ols(formula = 'patient_died_dummy ~ hospital_id',data = df_AB)
res_AB = model_AB.fit()
print(res_AB.summary())

                            OLS Regression Results                            
Dep. Variable:     patient_died_dummy   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.9377
Date:                Thu, 27 Jan 2022   Prob (F-statistic):              0.333
Time:                        10:07:26   Log-Likelihood:                -1446.8
No. Observations:                6611   AIC:                             2898.
Df Residuals:                    6609   BIC:                             2911.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.0970      0.005  

The difference in mortality rate implied by this regression cannot be interpreted as the causal effect of visiting a different hospital because we have non-experimental data where assignment is not completely random which could influence the effect of the independent variables in our regression. Therefore, we run the risk of having omitted variable bias and endogeneity in our model and prevents us from being able to assume causality within our regression since we haven't incorporated other relevant variables into our model.

### Question 2b.
*Do you think difference in mortality between hospitals are over- or under-estimated? Think about
what type of patients go to which type of hospital.*

Differences in mortality between hospitals could be both over or under-estimated depending on the types of observational differences that exist between hospitals. For example, some hospitals could have much older patients on average which could help with understanding why mortality rate may be higher and thus the mortality rate could appear to be over-estimated in that case.

### Question 2c.
*What are potential control variables that you might want to include in the regression, in order to
obtain a causal estimate (or at least get closer to a causal estimate)? Run such a regression with
suitable controls and interpret the change in the coefficient on the hospital B dummy. Explain why
you included the specific set of variables.*

In [29]:
model_AB_2 = smf.ols(formula = 'patient_died_dummy ~ hospital_id + startage + female_dummy',data = df_AB)
res_AB_2 = model_AB_2.fit()
print(res_AB_2.summary())

                            OLS Regression Results                            
Dep. Variable:     patient_died_dummy   R-squared:                       0.063
Model:                            OLS   Adj. R-squared:                  0.062
Method:                 Least Squares   F-statistic:                     147.6
Date:                Thu, 27 Jan 2022   Prob (F-statistic):           1.43e-92
Time:                        10:32:24   Log-Likelihood:                -1232.8
No. Observations:                6611   AIC:                             2474.
Df Residuals:                    6607   BIC:                             2501.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.1165      0.027  

In my new model, I decided to include control variables for age and gender as I believe that likely causes random variation in our non-fully randomized data which could help with estimating our mortality rates. As we can see, in our new regression the coefficient on the hospital B dummy has changed from 0.0072 to 0.0114. This change in the coefficient of the hospital B dummy demonstrates that the mortality rate difference from our original model was likely under-estimated due to not including in other control variables like age and gender that also influence our mortality rates.

## 2. Demand estimation
*The dataset demand data.csv contains data on sales and prices at a set of ice-cream vendors measured
over 52 weeks. All ice-cream at a given store is always priced the same, so there is only one price variable.
However, different vendors charge different prices and most vendors vary their prices throughout the year.*

### Question 1.
*Load demand data.csv into Python. For vendor 1, run a regression of sales on price and also
a regression of sales on price and a summer dummy (make sure your regression selects only the 52 weeks of
data for vendor 1). Use the omitted variable bias formula to explain why the price coefficient changes when
the summer dummy is also included in the regression.*

In [42]:
df_2 = pd.read_csv('demand_data.csv')
df_2.head()

,vendor_id,week,summer_dummy,price,sales
0,1,1,0,2.0,8788.7383
1,1,2,0,3.0,8937.9863
2,1,3,0,3.0,8740.1777
3,1,4,0,3.0,8757.1338
4,1,5,0,3.0,8739.6104


Obtain vendor 1 data below.

In [47]:
df_2_v1 = df_2[df_2['vendor_id'] == 1]
df_2_v1

,vendor_id,week,summer_dummy,price,sales
0,1,1,0,2.0,8788.7383
1,1,2,0,3.0,8937.9863
2,1,3,0,3.0,8740.1777
3,1,4,0,3.0,8757.1338
4,1,5,0,3.0,8739.6104
5,1,6,0,1.5,9094.7695
6,1,7,0,1.5,8657.3838
7,1,8,0,2.5,8677.9541
8,1,9,0,1.5,8641.8340
9,1,10,0,3.0,8626.8887


Run our first regression of sales on price for vendor 1 below.

In [49]:
model_ven1 = smf.ols(formula = 'sales ~ price',data = df_2_v1)
res_ven1 = model_ven1.fit()
print(res_ven1.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                 -0.013
Method:                 Least Squares   F-statistic:                    0.3250
Date:                Thu, 27 Jan 2022   Prob (F-statistic):              0.571
Time:                        11:15:57   Log-Likelihood:                -360.33
No. Observations:                  52   AIC:                             724.7
Df Residuals:                      50   BIC:                             728.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   8983.8227    145.437     61.771      0.0

Run our second regression of sales on price and a summer dummy of vendor 1 below.

In [51]:
model_ven1_sum = smf.ols(formula = 'sales ~ price + summer_dummy',data = df_2_v1)
res_ven1_sum = model_ven1_sum.fit()
print(res_ven1_sum.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.318
Model:                            OLS   Adj. R-squared:                  0.290
Method:                 Least Squares   F-statistic:                     11.42
Date:                Thu, 27 Jan 2022   Prob (F-statistic):           8.49e-05
Time:                        11:17:07   Log-Likelihood:                -350.56
No. Observations:                  52   AIC:                             707.1
Df Residuals:                      49   BIC:                             713.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     9177.5500    128.432     71.458   

The price coefficient changes when the summer dummy is also included in the regression because the omitted variable bias formula tells us that we are likely under-estimating the effect of price on our sales number. As we can see, including the summer dummy variable has a positive effect on our sales number which indicated why the magnitude of price effect has now increased.

### Question 2.
*Repeat the two regressions that you just ran in question 1, but now use data only for vendor 2.
In the case of the regression with the summer dummy, you should find that there might be multicollinearity
problems. Why does this happen?*

Obtain vendor 2 data below.

In [48]:
df_2_v2 = df_2[df_2['vendor_id'] == 2]
df_2_v2

,vendor_id,week,summer_dummy,price,sales
52,2,1,0,2.5,8976.3945
53,2,2,0,2.5,8523.9512
54,2,3,0,2.5,8847.0332
55,2,4,0,2.5,9055.2383
56,2,5,0,2.5,9053.9150
57,2,6,0,2.5,8724.5127
58,2,7,0,2.5,8870.8066
59,2,8,0,2.5,9571.6006
60,2,9,0,2.5,9081.9766
61,2,10,0,2.5,8631.9688


Run our first regression of sales on price for vendor 2 below.

In [52]:
model_ven2 = smf.ols(formula = 'sales ~ price',data = df_2_v2)
res_ven2 = model_ven2.fit()
print(res_ven2.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.133
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     7.684
Date:                Thu, 27 Jan 2022   Prob (F-statistic):            0.00781
Time:                        11:18:01   Log-Likelihood:                -359.10
No. Observations:                  52   AIC:                             722.2
Df Residuals:                      50   BIC:                             726.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   8411.1748    219.545     38.312      0.0

Run our second regression of sales on price and a summer dummy of vendor 1 below.

In [53]:
model_ven2_sum = smf.ols(formula = 'sales ~ price + summer_dummy',data = df_2_v2)
res_ven2_sum = model_ven2_sum.fit()
print(res_ven2_sum.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.133
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     7.684
Date:                Thu, 27 Jan 2022   Prob (F-statistic):            0.00781
Time:                        11:19:18   Log-Likelihood:                -359.10
No. Observations:                  52   AIC:                             722.2
Df Residuals:                      50   BIC:                             726.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     2105.3159     29.848     70.534   

There is a high degree of multicollinearity here because the inclusion of the summer dummy is directly correlated with the price variable. If we take a look at our data, we can see that each summer week was associated with a price increase to 3.5 while every other week that was not in the summer had a price of 2.5. This leads to multicollinearity and affects our ability to make effective predictions about sales for vendor 2.

### Question 3.
*Suppose that one of the vendors did not systematically charge higher or lower prices in
summer. If you were to repeat the analysis you just did for vendors 1 and 2, what would you expect to
happen to the price coefficient estimate and its precision in the two regressions with and without the summer
dummy?*

If I were to repeat the analysis I just did for vendors 1 and 2 without the vendors systematically charging higher or lower prices in the summer, I would expect the price coefficient estimate to not change as dramatically as before and have a similar value whether or not we include a dummy variable for the summer. This is due to the price and summer variables having little to no correlation with one another which means we eliminate omitted variable bias in our model. Although the price coefficient estimate will remain the same, adding in this control variable will still help with improving the precision of our variable in estimating sales.